In [1]:
from math import sin, cos, pi     #数学関数・定数のインポート
import numpy as np                #数値計算機能のインポート 別名np
import matplotlib.pyplot as plt   #グラフ描画機能のインポート 別名plt
from ipywidgets import interact   #対話的処理機能のインポート
from mpl_toolkits.mplot3d import Axes3D #3次元描画機能のインポート

# ロボット・マニピュレータ

### 同次変換行列(3次元)を作る関数

In [2]:
def H_3d(angle, axis, r):

  a1, a2, a3 = axis #回転軸の成分
  aa1 = a1*a1 #2乗を作っておく
  aa2 = a2*a2
  aa3 = a3*a3
  C = cos(angle)
  S = sin(angle)

  H = np.array([
    [ aa1 + (1-aa1)*C,    a1*a2*(1-C) - a3*S, a1*a3*(1-C) + a2*S, r[0] ],              
    [ a1*a2*(1-C) + a3*S, aa2 + (1-aa2)*C,    a2*a3*(1-C) - a1*S, r[1] ],              
    [ a1*a3*(1-C) - a2*S, a2*a3*(1-C) + a1*S, aa3 + (1-aa3)*C,    r[2] ],              
    [                  0,                  0,               0,      1  ],              
  ])

  return H

In [3]:
H_3d(pi/2, [1,0,0], [1,0,0]) #お試し

array([[ 1.000000e+00,  0.000000e+00,  0.000000e+00,  1.000000e+00],
       [ 0.000000e+00,  6.123234e-17, -1.000000e+00,  0.000000e+00],
       [ 0.000000e+00,  1.000000e+00,  6.123234e-17,  0.000000e+00],
       [ 0.000000e+00,  0.000000e+00,  0.000000e+00,  1.000000e+00]])

※ `6.123234e-17` $=6.123234\times10^{-17}$なので，計算機誤差を含む 0 のこと．

### ダミー成分を追加・除去する関数（3次元）

In [4]:
### 物理座標にダミー成分を追加する関数
def add_dummy(x):
#  return np.array([x[0], x[1], x[2], 1]) #ダミー成分を追加
  return np.append(x, 1) #同じ処理のPython的な書き方

### 同次座標からダミー変数を除去する関数
def remove_dummy(xx):
#  return np.array([xx[0], xx[1], xx[2]]) #ダミー成分を除去
  return xx[:-1] #同じ処理のPython的な書き方

In [5]:
H = H_3d(pi/2, [1,0,0], [1,2,3]) #お試し
x = np.array([1,2,3])
print(x)
hx = add_dummy(x)
print(hx)
hy = np.dot(H, hx) #同次変換行列と同次座標の積
print(hy)
y = remove_dummy(hy)
print(y)

[1 2 3]
[1 2 3 1]
[ 2. -1.  5.  1.]
[ 2. -1.  5.]


### テキストpart2.pdf の表2

In [6]:
def Trans(r):
  return H_3d(angle=0, axis=[1,0,0], r=r) #axis の方向は任意

Trans([1,2,3]) #お試し

array([[1., 0., 0., 1.],
       [0., 1., 0., 2.],
       [0., 0., 1., 3.],
       [0., 0., 0., 1.]])

In [7]:
def Rotx(angle):
  return H_3d(angle=angle, axis=[1,0,0], r=np.zeros(3))

Rotx(pi/3) #お試し

array([[ 1.       ,  0.       ,  0.       ,  0.       ],
       [ 0.       ,  0.5      , -0.8660254,  0.       ],
       [ 0.       ,  0.8660254,  0.5      ,  0.       ],
       [ 0.       ,  0.       ,  0.       ,  1.       ]])

In [8]:
def Roty(angle):
  return H_3d(angle=angle, axis=[0,1,0], r=np.zeros(3))

Roty(pi/3) #お試し

array([[ 0.5      ,  0.       ,  0.8660254,  0.       ],
       [ 0.       ,  1.       ,  0.       ,  0.       ],
       [-0.8660254,  0.       ,  0.5      ,  0.       ],
       [ 0.       ,  0.       ,  0.       ,  1.       ]])

In [9]:
def Rotz(angle):
  return H_3d(angle=angle, axis=[0,0,1], r=np.zeros(3))

Rotz(pi/3) #お試し

array([[ 0.5      , -0.8660254,  0.       ,  0.       ],
       [ 0.8660254,  0.5      ,  0.       ,  0.       ],
       [ 0.       ,  0.       ,  1.       ,  0.       ],
       [ 0.       ,  0.       ,  0.       ,  1.       ]])

## マニピュレータの同次変換

### 手先のデータ（今回は最初から同次座標で与える）

In [10]:
hand = np.array([
  [0,    0,    0,  1],  #[x,y,z,1]　1点目
  [0, -0.2,  0.2,  1],  #2点目
  [0,  0.2,  0.2,  1],  #3点目
  [0,    0,    0,  1]   #4点目 = 1点目
])

### マニピュレータの形状データを求める関数

In [11]:
def get_manipulator_points( hand, angles ):

  l1, l2, l3 = 0.8, 0.8, 0.8   #リンク長
  th1, th2, th3, th4 = angles  #関節角
  origin = np.array([0,0,0,1]) #原点の同時座標

  # 使用する同次変換行列
  R2 = Rotz( th4 )
  T3 = Trans([0,0,l3])
  R4 = Rotx( -th3 )
  T5 = Trans([0,0,l2])
  R6 = Rotx( -th2 )
  T7 = Trans([0,0,l1])
  R8 = Rotz( th1 )

  # 式(7.11)の形状データの作成

  ##手先　※行列の積np.dot(A,B)は，A.dot(B)という文法でも書けます 
  H = R8.dot(T7).dot(R6).dot(T5).dot(R4).dot(T3).dot(R2)
  xi1s = [] #空のリスト
  for point in hand:  #手先
    newpoint = np.dot(H, point)
    xi1s.append(newpoint) #計算結果をリストに追加
  xi1s = np.array(xi1s) #リストを配列化

  ##肘
  H = R8.dot(T7).dot(R6).dot(T5)
  xi4 = np.dot(H, origin)

  ##肩
  H = R8.dot(T7)
  xi5 = np.dot(H, origin)

  ##根本
  xi6 = origin

  ##一筆書きの点列
  allpoints = np.vstack((xi1s, xi4, xi5, xi6))

  ##ダミー成分(最終列)の除去
  allpoints = allpoints[:,:-1]

  return allpoints

get_manipulator_points(hand, [pi/2,pi/2,pi/2,pi/2]) #お試し

array([[-8.0000000e-01,  4.8985872e-17,  0.0000000e+00],
       [-8.0000000e-01,  2.0000000e-01, -2.0000000e-01],
       [-8.0000000e-01, -2.0000000e-01, -2.0000000e-01],
       [-8.0000000e-01,  4.8985872e-17,  0.0000000e+00],
       [-8.0000000e-01,  4.8985872e-17,  8.0000000e-01],
       [ 0.0000000e+00,  0.0000000e+00,  8.0000000e-01],
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00]])

## 手動操作

In [14]:
@interact(
    th1deg=(-180, 180, 10), #[下限，上限，刻み幅]
    th2deg=(-180, 180, 10),
    th3deg=(-180, 180, 10),
    th4deg=(-180, 180, 10),
)
def f(th1deg=-40, th2deg=50, th3deg=30, th4deg=90):

    print('angles = (%d, %d, %d, %d)'%(th1deg, th2deg, th3deg, th4deg))

    angles_rad = np.array([th1deg, th2deg, th3deg, th4deg])/180*pi 
    newpoints = get_manipulator_points( hand, angles=angles_rad )
    xs = newpoints[:,0] #1列目　x座標
    ys = newpoints[:,1] #2列目　y座標
    zs = newpoints[:,2] #3列目　z座標

    fig = plt.figure(figsize=(8,8))   #正方形の用紙
    ax = plt.axes(projection='3d')    #用紙上に3次元グラフ用紙を作る
    ax.set_xlim([-0.5, 2])            #x軸の範囲
    ax.set_ylim([-0.5, 2])            #y軸の範囲
    ax.set_zlim([   0, 2.5])          #z軸の範囲
    ax.set_xlabel('x')                #x軸のラベル
    ax.set_ylabel('y')                #y軸のラベル
    ax.set_zlabel('z')                #x軸のラベル
    ax.grid()

    ax.plot(xs, ys, zs)

interactive(children=(IntSlider(value=-40, description='th1deg', max=180, min=-180, step=10), IntSlider(value=…